## 1. 모듈 Import

In [ ]:
import pythoncom
import os, time, io, pptx
import pandas as pd

from pptxtopdf import convert
from pptx import Presentation

import mysql.connector
import yaml
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature
from azure.ai.documentintelligence.models import ContentFormat
import re, pdfplumber
import tiktoken
from openai import AzureOpenAI
import json, time, base64
import fitz
from tqdm import tqdm
import pickle, requests
from collections import defaultdict, Counter
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions, ContentSettings
from azure.core.exceptions import *

from openpyxl import load_workbook
from openpyxl.utils.cell import range_boundaries
from copy import copy
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

from datetime import datetime, timedelta, timezone
from urllib.request import urlopen

import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# 임시 함수들
from temp_functions import *

## 2. config 정보 연결

In [ ]:
with open('configs\\config.yml', 'r', encoding='UTF-8') as f:
    config = yaml.safe_load(f)

mysql_config = config['mysql_dev']   # 개발 DB 연결 정보
blob_config  = config['blob_storage_dev']   # 개발 Blob Storage 연결 정보
blob_config_prod2  = config['blob_storage_prod2']   # 운영 Blob Storage 연결 정보
di_config = config['di']   # Document Intelligence 연결 정보
aoai_config  = config['aoai']['gpt_4_0409']   # GPT 리소스 정보
ss_vars      = config['section_split_vars']
gpt_split_vars = ss_vars['gpt_split']
blob_service_client  = BlobServiceClient.from_connection_string(blob_config['connect_string'])
blob_service_client_prod2  = BlobServiceClient.from_connection_string(blob_config_prod2['connect_string'])

DI_ENDPOINT = di_config['endpoint']
DI_KEY      = di_config['key']
DI_API_VERSION = di_config['api_version']

document_intelligence_client = DocumentIntelligenceClient(
        endpoint    = DI_ENDPOINT,
        credential  = AzureKeyCredential(DI_KEY),
        api_version = DI_API_VERSION
    )

## 3. 전처리 함수 정의

In [ ]:
prep_container_client = blob_service_client.get_container_client(container="prep-data")


## HTML SAS URL 생성 함수
def get_sas_url(blob_config, container_client, blob_path, expiry_time=True) :

    blob_client = container_client.get_blob_client(blob_path)
    if expiry_time :
        kst = timezone(timedelta(hours=9))
        start_time_kst = datetime.now(kst) - timedelta(minutes=5)

        expiry_time_kst = start_time_kst + timedelta(days=1)
        start_time_utc = start_time_kst.astimezone(timezone.utc)
        expiry_time_utc = expiry_time_kst.astimezone(timezone.utc)
    
    else :
        kst = timezone(timedelta(hours=9))
        start_time_kst = datetime.now(kst) - timedelta(minutes=5)
        start_time_utc = start_time_kst.astimezone(timezone.utc)
        expiry_time_utc = datetime(9999, 12, 31)

    try: 
        sas_token = generate_blob_sas(
            account_name=blob_config['account_name'],
            account_key=blob_config['account_key'],
            container_name=container_client.container_name,
            blob_name=blob_path,
            permission=BlobSasPermissions(read=True, write=True, delete=True, add=True, create=True, update=True, process=True),
            start=start_time_utc,
            expiry=expiry_time_utc                  
        )
        sas_url = blob_client.url + '?' + sas_token
        return sas_url

    except Exception as e:
        sas_url = None  
        return None
    

## 현재 섹션 분할 대상 페이지의 Paragraphs만 추출하는 함수
def get_page_paragraphs (paragraphs, pageNum, except_texts) :
    pageParagraphs = []
    
    for para in paragraphs :
        if para.bounding_regions[0]['pageNumber'] == pageNum :
            content = para.content
            
            for except_text in except_texts:
                if except_text in content:
                    content = content.replace(except_text, '')
            
            content = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', content)
            content = content.replace(' ', '').replace('\n', '').replace('\\', '') \
                    .replace('/', '').strip()
                    
            if 'Q봇학습제외' in content:
                return None
            
            pageParagraphs.append(para)
            
    return pageParagraphs


## 현재 섹션 분할 대상 페이지의 Table을 Markdown으로 변환하는 함수
def get_page_table (tables, pageNum, header_x=0.0) :
    txtMarkdown = f""
    tablesMD = []
    cells_offset = []

    for table in tables :
        if table['boundingRegions'][0]['polygon'][0] > header_x:
            tabe_pageNum = table.get("boundingRegions", [])[0].get("pageNumber")
            if tabe_pageNum == pageNum :
                txtMarkdown = f"|"
                totRowCnt = table.row_count
                totColCnt = table.column_count

                preRow = 0
                for rIdx in range(totRowCnt) :
                    if preRow != rIdx :
                        txtMarkdown = txtMarkdown + "\n|"
                    for cIdx in range(totColCnt) :
                        cell = next((cell for cell in table['cells'] if cell['rowIndex'] == rIdx and cell['columnIndex'] == cIdx), None)
                        if cell != None :
                            content = cell.content.replace('\n', '')
                            txtMarkdown = txtMarkdown + f"{content}"
                            if cell['spans'] != [] :
                                cells_offset.append(cell['spans'][0]['offset'])
                                
                        txtMarkdown = txtMarkdown + "|"
                        
                txtMarkdown = txtMarkdown + f"\n"
                
                tablesMD.append({'pageNumber':tabe_pageNum, 'spans':table.spans, 'cells':table.cells,'boundingRegions':table.bounding_regions,'markdown':txtMarkdown})
            
    return tablesMD, cells_offset


## Blob Storage에 섹션명으로 된 폴더가 있는 확인하는 함수
def folder_exists_in_azure(base_path, section_name):
    # 폴더명 리스트 가져오기 위해 blob 이름 확인
    folder_names = set()
    blobs = prep_container_client.list_blobs(name_starts_with=base_path)
    for blob in blobs:
        blob_name = blob.name[len(base_path):]  # base_path 이후의 부분만 확인
        parts = blob_name.split('/')  # '/' 기준으로 분할
        if len(parts) > 1:  # 2개 이상이면 하위 폴더를 의미
            folder_names.add(parts[0])

    # section_name과 일치하는 폴더가 있는지 확인
    return any(name for name in folder_names if name == section_name), folder_names


## Table을 데이터프레임으로 변환하는 함수
def generate_dataframe_from_table(table):
    # 2차원 배열 생성하여 각 셀 저장 공간 마련
    rows = [["" for _ in range(table['columnCount'])] for _ in range(table['rowCount'])]

    # 각 셀 순회하며 content 채우기
    for cell in table['cells']:
        row = cell['rowIndex']
        col = cell['columnIndex']
        content = cell['content']
        row_span = cell.get('rowSpan', 1)
        col_span = cell.get('columnSpan', 1)

        # rowSpan과 columSpan을 순회하며 content 채우기
        for r in range(row, row + row_span):
            for c in range(col, col + col_span):
                rows[r][c] = content

    # 데이터프레임으로 변환
    df = pd.DataFrame(rows)
    
    return df


## 데이터프레임을 Markdown으로 변환하는 함수
def save_df_to_markdown(df):
    # 첫 번째 행은 헤더로 설정, 나머지는 테이블로 변환
    rows = df.values.tolist()
    if not rows:
        markdown_str = ''
    else:
        # 헤더와 데이터 구분
        headers = rows[0]
        data = rows[1:]
       
        # Markdown 테이블 생성
        markdown_str = '| ' + ' | '.join(str(header) for header in headers) + ' |\n'
        for row in data:
            markdown_str += '| ' + ' | '.join(str(cell) for cell in row) + ' |\n'
   
    return markdown_str


## 4. 텍스트 추출 수행

In [ ]:
query = f"""
SELECT *
FROM TB_PREP_SOURCE
WHERE 1=1
    AND IF_CATEGORY_CD = '핸드북'
    AND FILE_NM LIKE '%서비스 핸드북%'
    AND PREP_TYPE_CD = 'PAGEHEADER'
    AND PREP_STATUS_CD = 'N'
;
"""

tmp_df = select_table(mysql_config, query)
tmp_df['DI_RUNNING_TIME'] = None  # 실행시간 컬럼 추가


raw_container_client = blob_service_client.get_container_client(container="raw-data")
pkl_container_client = blob_service_client.get_container_client(container="pkl-data")

# pptx를 pdf로 변환 및 pkl 업로드에 필요한 로컬 폴더 생성
os.makedirs('DI', exist_ok=True)
os.makedirs('pkl', exist_ok=True)

for i,row in tmp_df.iterrows():
    file_path = row['SOURCE_FILE_PATH']
    file_name = row['FILE_NM']
    file_name_without_ext = os.path.splitext(file_name)[0]
    file_ext = os.path.splitext(file_name)[1]

    # pkl 폴더의 모든 파일 삭제
    for filename in os.listdir('pkl'):
        pkl_path = os.path.join('pkl', filename)
        try:
            if os.path.isfile(pkl_path):
                os.unlink(pkl_path)
        except Exception as e:
            print(f"Failed to delete {pkl_path}. Reason: {e}")
    
    print('##############################################################')
    print(datetime.now())
    print(f"{i} : {file_path}/{file_name}")
    print('##############################################################')
    
    # 엑셀이면 패스
    if file_ext == '.xlsx' or file_ext == '.xls':
        continue
    
    # 확장자가 pptx면 pdf로 변환 후 저장
    if file_ext == '.pptx' or file_ext == '.ppt':
        pythoncom.CoInitialize()
        
        for filename in os.listdir('DI'):
            pdf_path = os.path.join('DI', filename)
            try:
                if os.path.isfile(pdf_path):
                    os.unlink(pdf_path)
            except Exception as e:
                print(f"Failed to delete {pdf_path}. Reason: {e}")
        
        try:
            file_client = raw_container_client.get_blob_client(blob=f"{file_path}/{file_name}")
            
            # PPTX 파일을 로컬에 다운로드
            with open(f"DI\\{file_name}", "wb") as file:
                file.write(file_client.download_blob().readall())
                
            # PPTX 파일을 PDF로 변환 후 저장
            convert(f"DI\\{file_name}", f"{os.getcwd()}\\DI")
            
        except Exception as e:
            print(e)
            print('pass')
        
        try:
            pdf_file_client = raw_container_client.get_blob_client(blob=f"{file_path}/{file_name_without_ext}.pdf")
            with open(f"DI\\{file_name_without_ext}.pdf", "rb") as f:
                pdf_file_client.upload_blob(f, overwrite=True, content_settings=ContentSettings(content_type="application/pdf"))
                
            file_name = f"{file_name_without_ext}.pdf"
        except Exception as e:
            print(e)
            continue
        
        pythoncom.CoUninitialize()
    
    try:
        start_time = time.time()

        sas_url = get_sas_url(blob_config, container_client=raw_container_client, blob_path=f'{file_path}/{file_name}', expiry_time=False)

        with urlopen(sas_url) as f:
            pdf_data = f.read()
            
            # DI 사용하는 경우 
            poller = document_intelligence_client.begin_analyze_document(
                "prebuilt-layout",
                analyze_request=pdf_data,
                features=[DocumentAnalysisFeature.KEY_VALUE_PAIRS],# , DocumentAnalysisFeature.LANGUAGES],
                content_type="application/octet-stream",
                output_content_format = ContentFormat.MARKDOWN # ContentFormat.TEXT 
            )

        result = poller.result()
        
        end_time = time.time()
        
        execution_time_seconds = end_time - start_time  # 소스코드 실행 시간 계산 (초 단위)
        print(execution_time_seconds)
        print(file_name)
        
        tmp_df.loc[i, 'DI_RUNNING_TIME'] = execution_time_seconds
            
        with open(f"pkl\\{file_name_without_ext}.pkl", "wb") as f: # 한 번 돌려서 pickle 파일로 저장
            pickle.dump(result, f)
            
        blob_client = pkl_container_client.get_blob_client(blob=f"{file_path}/{file_name_without_ext}.pkl")
        
        with open(f"pkl\\{file_name_without_ext}.pkl", "rb") as data:
            blob_client.upload_blob(data, overwrite=True, content_settings=ContentSettings(content_type="application/octet-stream"))
            
        update_query = f"""
            UPDATE TB_PREP_SOURCE
            SET PREP_STATUS_CD = 'C'
                , UPDATE_PGM = 'PAGEHEADER 전처리'
                , UPDATE_DTTM = SYSDATE()
            WHERE 1=1
                AND IF_CATEGORY_CD = '{row['IF_CATEGORY_CD']}'
                AND CONTENTS_ID    = '{row['CONTENTS_ID']}'
                AND ITEM_ID        = '{row['ITEM_ID']}'
            ;
        """
        commit_query(mysql_config, update_query)
        print('Compelete!!!!!!!!!!!!!!!!')
            
    except Exception as e:
        print(f"{str(e)}")
        
        error_msg = f"{str(e)}"
        
        update_query = f"""
            UPDATE TB_PREP_SOURCE
            SET PREP_STATUS_CD = 'E'
                , PREP_ERROR_MSG = '{error_msg}'
                , UPDATE_PGM = 'PAGEHEADER 전처리'
                , UPDATE_DTTM = SYSDATE()
            WHERE 1=1
                AND IF_CATEGORY_CD = '{row['IF_CATEGORY_CD']}'
                AND CONTENTS_ID    = '{row['CONTENTS_ID']}'
                AND ITEM_ID        = '{row['ITEM_ID']}'
            ;
        """
        commit_query(mysql_config, update_query)
        continue
    
    time.sleep(3)

## 5. 섹션 분할 수행

In [ ]:
pdf_container_client = blob_service_client.get_container_client(container="raw-data")
prep_container_client = blob_service_client.get_container_client(container="prep-data")
web_client = blob_service_client_prod2.get_container_client(container="$web")


## 페이지별 pageheader 추출할 때 제외할 날짜 텍스트 입력
except_date = '20240829'


## 전처리 대상 load 
query = f"""
SELECT 
    *
FROM TB_PREP_SOURCE
WHERE 1=1
    AND IF_CATEGORY_CD = '핸드북'
    AND FILE_NM LIKE '%서비스 핸드북%'
    AND PREP_TYPE_CD = 'PAGEHEADER'
    AND PREP_STATUS_CD = 'C'
;
"""
tb_if_prep_files_source = select_table(mysql_config, query)

## pkl 블롭 파일 목록
pkl_container_client = blob_service_client.get_container_client(container="pkl-data")
pkl_blob_list = [i for i in pkl_container_client.list_blob_names() if ".pkl" in i]
## 테이블 활용해서 pkl 파일 위치 찾기
pkl_path_df = tb_if_prep_files_source[['IF_CATEGORY_CD', 'CONTENTS_ID', 'ITEM_ID'
                                       ,'FILE_NM', 'SOURCE_FILE_PATH', 'RESULT_FILE_PATH'
                                       , 'PRODUCT_LVL1_CD', 'PRODUCT_LVL2_CD', 'PRODUCT_LVL3_CD'
                                       , 'PROD_MODEL_CD', 'SALES_MODEL_CD']].drop_duplicates()

## 제외 Text load
query = f"""
SELECT TRIM(EXCEPT_TEXT_VAL) AS EXCEPT_TEXT_VAL
FROM TB_EXCEPT_TEXT_MST
WHERE USE_FLAG = 'Y'
ORDER BY LENGTH(EXCEPT_TEXT_VAL) DESC
;
"""
TB_MST_PREP_EXCEPT_TEXT = select_table(mysql_config, query)
except_texts = TB_MST_PREP_EXCEPT_TEXT['EXCEPT_TEXT_VAL'].to_list()

pkl_path_df['CT_SECTION_SPLIT_RUNNING_TIME'] = None


# 파일 하나씩 loop
for idx in tqdm(pkl_path_df.index):
    error_msg = ''
    
    idx_row  = pkl_path_df.loc[idx]
    category_cd = idx_row['IF_CATEGORY_CD']
    contents_id = idx_row['CONTENTS_ID']
    item_id = idx_row['ITEM_ID']
    prod_model_cd = idx_row['PROD_MODEL_CD']
    sales_model_cd = idx_row['SALES_MODEL_CD']
    
    file_nm  = idx_row['FILE_NM']
    file_nm_no_ext = os.path.splitext(file_nm)[0]
    file_type = file_nm.split('.')[-1]

    src_fpath = idx_row['SOURCE_FILE_PATH']
    result_path = idx_row['RESULT_FILE_PATH']

    lvl1_cd = idx_row['PRODUCT_LVL1_CD']
    lvl2_cd = idx_row['PRODUCT_LVL2_CD']
    lvl3_cd = idx_row['PRODUCT_LVL3_CD']

    print('##############################################################')
    print(f'################### {src_fpath}/{file_nm} ###################')
    print('##############################################################')
    
    cur_pkl_blob_list = [pkl_blob for pkl_blob in pkl_blob_list if f"{src_fpath}/{file_nm_no_ext}.pkl" == pkl_blob]
    if (len(cur_pkl_blob_list) == 0) | (len(cur_pkl_blob_list) > 1) :
        print(f"pickle 파일이 없거나 너무 많습니다.")
        print(f"PKL FILE LIST : {cur_pkl_blob_list}")
        continue
    
    ## Load Pkl
    pkl_blob_client = pkl_container_client.get_blob_client(f"{src_fpath}/{file_nm_no_ext}.pkl")
    pkl_db_bytes   = pkl_blob_client.download_blob().readall()
    pkl_blob_stream = io.BytesIO(pkl_db_bytes)
    di_result = pickle.load(pkl_blob_stream)
    
    
    try:
        sas_url = get_sas_url(blob_config, container_client=pdf_container_client, blob_path=f"{src_fpath}/{file_nm_no_ext}.pdf", expiry_time=False)
        request_blob = requests.get(sas_url)
        filestream = io.BytesIO(request_blob.content)
        doc = pdfplumber.open(filestream)
        doc2 = fitz.open(stream=filestream, filetype="pdf")
    except:
        sas_url = get_sas_url(blob_config, container_client=pdf_container_client, blob_path=f"{src_fpath}/{file_nm_no_ext}.PDF", expiry_time=False)
        request_blob = requests.get(sas_url)
        filestream = io.BytesIO(request_blob.content)
        doc = pdfplumber.open(filestream)
        doc2 = fitz.open(stream=filestream, filetype="pdf")


    # size_min, size_max를 가져오기 위해 RESULT 테이블 SELECT
    query = f"""
        SELECT 
            SEQ_ID, SIZE_MIN, SIZE_MAX
        FROM TB_PREP_RESULT
        WHERE 1=1
            AND IF_CATEGORY_CD = '{idx_row['IF_CATEGORY_CD']}'
            AND FILE_NM LIKE '%서비스 핸드북%'
            AND IF_TYPE_CD != 'D'
        ;
    """
    handbook_size_df = select_table(mysql_config, query)


    # 결과를 insert할 데이터프레임 구조 정의
    query = f"""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'TB_PREP_RESULT_STG'
    AND TABLE_SCHEMA = 'QBOTDB'
    ORDER BY ORDINAL_POSITION ASC;
    """
    result_df = select_table(mysql_config, query)
    result_list = result_df['COLUMN_NAME'].to_list()
    df_data = {col: [None]*len(doc.pages) for col in result_list}
    result_df = pd.DataFrame(df_data)
    
    # result_df에 고정값 세팅
    result_df['IF_CATEGORY_CD'] = category_cd
    result_df['CONTENTS_ID'] = contents_id
    result_df['ITEM_ID'] = item_id
    result_df['FILE_NM'] = file_nm_no_ext
    result_df['MODEL_USE_FLAG'] = 'Y'
    result_df['PRODUCT_LVL1_CD'] = lvl1_cd
    result_df['PRODUCT_LVL2_CD'] = lvl2_cd
    result_df['PRODUCT_LVL3_CD'] = lvl3_cd
    result_df['PROD_MODEL_CD'] = prod_model_cd
    result_df['SALES_MODEL_CD'] = sales_model_cd
    result_df['IF_FLAG'] = 'N'
    result_df['CREATE_DTTM'] = datetime.now()
    result_df['CREATE_PGM'] = 'PAGEHEADER 전처리'
    result_df['IF_TYPE_CD'] = 'U'
    
    # 기존에 해당 파일을 전처리 했으면 blob에서 삭제
    result_file_blobs = prep_container_client.list_blobs(name_starts_with=f'{result_path}/{file_nm_no_ext}/')
    for result_file_blob in result_file_blobs:
        result_blob_client = prep_container_client.get_blob_client(result_file_blob)
        result_blob_client.delete_blob()

    result_df_idx = 0
     
    prc_str_time = time.time()
    
    # 페이지별로 loop
    for page in di_result.pages:
        section_file_name = None
        saveText = ""
        saveText_bytes = None
        saveText_base64 = None
        size_min = None
        size_max = None
        pageContents = []

        table_markdown_flag = False

        tables = []
        cells_offset = []  
        
        pageHeader = None      # PageHeader
        title = None           # title
        sectionHeading = None  # SectionHeading
        first_text = None      # 첫 번째로 추출되는 문자
        
        pageNum = page.page_number
        pageParagraphs = get_page_paragraphs(di_result.paragraphs, pageNum, except_texts)
        
        print('==============================================================')
        print(f'================== 현재 pageNum : {pageNum} ==================')
        print('==============================================================')
        
        if pageParagraphs == [] or pageParagraphs == None:
            continue
            
        # PageHeader 추출
        for para in pageParagraphs:
            page_content = para.content
            
            ## 제외 Text 제외
            for except_text in except_texts:
                if except_text in page_content:
                    page_content = page_content.replace(except_text, '')
            
            page_content = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', page_content)
            page_content = page_content.replace('\n', '').replace('\\', '') \
                    .replace('/', '').strip()

            page_content_chk = page_content.replace(' ', '')

            if '부표' in page_content_chk or except_date in page_content_chk:
                continue

            if '수리유형' in page_content_chk or '수리코드' in page_content_chk:
                table_markdown_flag = True

            ## Pageheader 추출
            if page_content_chk != '' :
                if para.role == 'pageHeader':
                    if pageHeader == None :
                        pageHeader = page_content
                if para.role == 'title':
                    if title == None :
                        title = page_content
                if para.role == 'sectionHeading':
                    if sectionHeading == None :
                        sectionHeading = page_content
                if first_text == None :
                    first_text = page_content
                    first_text = first_text[:990]
        
        if pageHeader != None :    # PageHeader가 잡힌 경우
            section_file_name = pageHeader
        elif title != None :       # title이 잡힌 경우
            section_file_name = title
        elif sectionHeading != None :  # sectionHeading이 잡힌 경우
            section_file_name = sectionHeading
        elif first_text != None :  # 첫 번째로 문자가 추출된 경우
            section_file_name = first_text
        section_file_name = section_file_name[:990]
        
        # 섹션 분할
        pageContents.append(f"{pageNum} : {section_file_name}")
        
        if di_result.tables != None:
            tables, cells_offset = get_page_table(di_result.tables, pageNum)

            table_idx = 0
            for table in di_result.tables:
                convert_df = None
                new_markdown = None
                
                tabe_pageNum = table.get("boundingRegions", [])[0].get("pageNumber")

                if tabe_pageNum != pageNum:
                    continue
                
                convert_df = generate_dataframe_from_table(table)

                if section_file_name.startswith('제품별 수리코드') and table_markdown_flag:
                    # 공백인 셀의 행, 열 좌표 추출
                    empty_cells = [(i, j) for i in range(len(convert_df)) for j in [3, 4, 5] if convert_df.iloc[i, j] == '']

                    # 동일한 수리유형과 수리코드를 가진 다른 행에서 값 추출 및 매핑
                    for i, j in empty_cells:
                        repair_type = convert_df.iloc[i, 0]
                        repair_code = convert_df.iloc[i, 1]

                        # 공백이 없는 행들 중 동일한 수리유형과 수리코드를 가진 행 찾기
                        matching_rows = convert_df[
                            (convert_df[0] == repair_type) & 
                            (convert_df[1] == repair_code) & 
                            (convert_df.index != i) & 
                            (convert_df.iloc[:, j] != '')  # 해당 열에 공백이 없는 행만 선택
                        ]

                        if not matching_rows.empty:
                            # j열의 값을 매핑
                            convert_df.iloc[i, j] = matching_rows.iloc[0, j]
        
                new_markdown = save_df_to_markdown(convert_df)
                tables[table_idx]['markdown'] = new_markdown

                table_idx += 1

        
        for para in pageParagraphs :
            if para['spans'] != []:
                para_offset = para['spans'][0]['offset']
                
                if len(tables) > 0 :
                    if tables[0]['spans'] != []:
                        table_offset = tables[0]['spans'][0]['offset']
                        if para_offset >= table_offset :
                            pageContents.append(tables[0]['markdown'])
                            del tables[0]
                    
                if para_offset not in cells_offset:
                    pageContents.append(para['content'])
                    
        for i in range(len(pageContents)) :
            content = pageContents[i]
            
            for except_text in except_texts:
                if except_text in content:
                    content = content.replace(except_text, '')
                    
            saveText = saveText + content + '\n'
            pageContents[i] = content
        
        ## Blob 저장 (저장할 Text)
        exists, folder_names = folder_exists_in_azure(f"{result_path}/{file_nm_no_ext}/", section_file_name)
        
        if exists:
            folder_name = f'{result_path}/{file_nm_no_ext}/' + [name for name in folder_names if name == section_file_name][0]
        else:
            folder_name = f"{result_path}/{file_nm_no_ext}/{section_file_name}"
            
        save_blob_name = f"{folder_name}/{str(pageNum).zfill(3)}_{section_file_name}.txt"
        print(save_blob_name)
        
        ## result_df에 필요한 정보 추가
        result_df.loc[result_df_idx, 'SECTION_NM'] = folder_name.split('/')[-1]
        result_df.loc[result_df_idx, 'SECTION_FILE_NM'] = save_blob_name.split('/')[-1]
        result_df.loc[result_df_idx, 'FULL_FILE_PATH'] = save_blob_name
        result_df.loc[result_df_idx, 'SEQ_ID'] = pageNum
        result_df.loc[result_df_idx, 'PAGE_NUM'] = pageNum
        result_df.loc[result_df_idx, 'PAGE_NUMS'] = pageNum

        # 해당 seq_id에 해당하는 SIZE_MIN, SIZE_MAX 가져오기
        size_result = handbook_size_df.loc[handbook_size_df['SEQ_ID'] == pageNum, ['SIZE_MIN', 'SIZE_MAX']]

        if not size_result.empty:
            size_min = size_result.iloc[0]['SIZE_MIN']
            size_max = size_result.iloc[0]['SIZE_MAX']

            # SIZE_MIN, SIZE_MAX 설정
            result_df.loc[result_df_idx, 'SIZE_MIN'] = size_min
            result_df.loc[result_df_idx, 'SIZE_MAX'] = size_max
            
        # 각 페이지 txt 파일 적재  --> Blob 에 적재
        blob_client = prep_container_client.get_blob_client(blob=save_blob_name)
        blob_client.upload_blob(saveText, overwrite=True)
        
        saveText_bytes = saveText.encode('utf-8')
        saveText_base64 = base64.b64encode(saveText_bytes).decode('utf-8')
        result_df.loc[result_df_idx, 'RESULT_TEXT'] = saveText_base64
        
        # 각 페이지에서 image가 차지하는 비율 계산
        target = doc.pages[pageNum-1]
        
        text_length_sum = 0
        image_sum = 0
        image_ratio = 0
        
        # 페이지 넓이 계산
        page_area = round(target.width*target.height, 2)
        
        # 텍스트 개수 계산
        for texts in target.extract_words():
            text = texts['text']
            length = len(text)
            text_length_sum += length
            
        result_df.loc[result_df_idx, 'TEXT_CNT'] = text_length_sum
        
        if target.images != []:
            # Image 넓이 계산
            for images in target.images:
                width = round(images['width'], 2)
                height = round(images['height'], 2)
                area = round(width*height, 2)
                image_sum += area
            image_sum = round(image_sum, 2)
            image_ratio = round(image_sum / page_area, 2)
            
            result_df.loc[result_df_idx, 'IMAGE_RATIO'] = image_ratio

            caption_start = []
            pad = 15
            dpi = 200
            seq = 1
            
            # 캡션 텍스트 좌표 확인
            for text in target.extract_words():
                if '[이미지_' in text['text']:
                    caption_start.append([text['x0'], text['bottom'], text['text']])
            
            try:
                # 페이지별 이미지 추출
                for image in target.images:
                    for i in range(len(caption_start)):
                        start_x = caption_start[i][0]
                        start_y = caption_start[i][1]
                        tag_name = caption_start[i][2]
                        
                        if (start_x-pad <= image['x0'] <= start_x+pad) and (start_y <= image['top'] <= start_y+pad):
                            page = doc2[pageNum-1]
                            page.set_cropbox([image['x0'], image['top'], image['x1'], image['bottom']])
                            pixmap = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
                            # PNG 형식으로 이미지 데이터 추출
                            img_data = pixmap.tobytes("png")
                            # Blob Storage에 업로드
                            img_blob_client = prep_container_client.get_blob_client(f"{folder_name}/images/{str(pageNum).zfill(3)}_{str(seq).zfill(3)}_{tag_name}.png")
                            img_blob_client.upload_blob(img_data, overwrite=True, content_type="image/png")
                            
                            seq += 1
            except Exception as e:
                print(f"{str(e)}")
                
                result_df_idx += 1
                continue
                    
        result_df_idx += 1

    doc2.close()
    
    ## PAGE_NUM이 NULL인 행 삭제
    result_df = result_df.dropna(subset=['PAGE_NUM'])

    # NaN을 None으로 변환
    result_df = result_df.replace({np.nan: None})
     
    ## HTML 생성
    try:
        sas_url = get_sas_url(blob_config, container_client=pdf_container_client, blob_path=f"{src_fpath}/{file_nm_no_ext}.pdf", expiry_time=False)
        request_blob = requests.get(sas_url)
        filestream = io.BytesIO(request_blob.content)
        doc = fitz.open(stream=filestream, filetype="pdf")
    except:
        sas_url = get_sas_url(blob_config, container_client=pdf_container_client, blob_path=f"{src_fpath}/{file_nm_no_ext}.PDF", expiry_time=False)
        request_blob = requests.get(sas_url)
        filestream = io.BytesIO(request_blob.content)
        doc = fitz.open(stream=filestream, filetype="pdf")
    print(f"[#] 현재 문서 : {file_nm_no_ext}.pdf")
    print(f"page 수 : {len(doc)}")

    # pdf 저장 경로 사용해서 적재하는경우
    save_folder = f'{result_path}/{file_nm_no_ext}'

    for i, page in tqdm(enumerate(doc)):
        svg = page.get_svg_image(matrix=fitz.Identity)
        svg_bytes  = svg.encode('utf-8')
        img_data = io.BytesIO(svg_bytes)
        img_data.seek(0)
        img_blob_client = web_client.get_blob_client(f"{save_folder}/images/{i+1}.svg")
        img_blob_client.upload_blob(img_data, content_type="image/svg+xml", overwrite=True)
    doc.close()

    img_path       = f"{save_folder}/images/"
    img_bname_list = web_client.list_blobs(name_starts_with = img_path) # 적재된 image 목록 
    img_bname_list = [i.name for i in img_bname_list if i.name.endswith('.svg')]

    if len(img_bname_list) == 0:
        print("적재된 Image 파일 없음")
    else : 
        blob_client = web_client.get_blob_client(f'{result_path}/{file_nm_no_ext}')
        root_folder = f"{blob_client.url}/images"

        html_content = "<html><head></head><body style='background-color: white;'></body>\n"
        html_content += "<script>\n"
        
        if file_type == 'ppt' or file_type == 'pptx':
            html_content += f"""
const rootFolder = '{root_folder}'
const sas_key = 'sv=2023-11-03&st=2024-07-24T08%3A08%3A12Z&se=9999-12-31T12%3A00%3A00Z&sr=c&sp=rl&sig=vfHfN9ax60XVqxsZ5ae2VUjAomSwqZGKOLrptQiDKko%3D'
const hash = window.location.hash;
const page = hash.match(/#page(\d+)/)[1];

if(page) {{
    let pageInt = parseInt(page)
    let startPage = pageInt - 5;
    let endPage = pageInt + 5;

    let body = document.body;
    for ( i = startPage; i <= endPage; i++) {{

        if(i >= 1){{
            
            div = document.createElement('div');
            div.setAttribute('id', 'page'+i);
            div.setAttribute('style', 'text-align: center;');

            let img = document.createElement('img');
            let src = rootFolder+'/'+i+'.svg'+'?'+sas_key
            img.setAttribute('src', src)
            img.setAttribute('style', 'display: inline-block; margin:10px auto;width:1500px;');
            img.onerror = function() {{
                img.remove();
            }};
            
            if(img) {{
                div.appendChild(img);
                body.appendChild(div);
            }}
        }}
    }}
}}
            """

        else:
            
            html_content += f"""
const rootFolder = '{root_folder}'
const sas_key = 'sv=2023-11-03&st=2024-07-24T08%3A08%3A12Z&se=9999-12-31T12%3A00%3A00Z&sr=c&sp=rl&sig=vfHfN9ax60XVqxsZ5ae2VUjAomSwqZGKOLrptQiDKko%3D'
const hash = window.location.hash;
const page = hash.match(/#page(\d+)/)[1];

if(page) {{
    let pageInt = parseInt(page)
    let startPage = pageInt - 5;
    let endPage = pageInt + 5;

    let body = document.body;
    for ( i = startPage; i <= endPage; i++) {{

        if(i >= 1){{
            
            div = document.createElement('div');
            div.setAttribute('id', 'page'+i);
            div.setAttribute('style', 'text-align: center;');

            let img = document.createElement('img');
            let src = rootFolder+'/'+i+'.svg'+'?'+sas_key
            img.setAttribute('src', src)
            img.setAttribute('style', 'display: inline-block; margin:10px auto;width:70%;');
            img.onerror = function() {{
                img.remove();
            }};
            
            if(img) {{
                div.appendChild(img);
                body.appendChild(div);
            }}
        }}
    }}
}}
            """

        html_content += "\n</script></html>"
            
        html_bname = f"{save_folder}/{file_nm_no_ext}.html"
        html_blob_client = web_client.get_blob_client(html_bname)
        html_blob_client.upload_blob(html_content, content_type=("text/html"), overwrite=True)
        
        ## SAS Link 생성
        html_sas_url = get_sas_url(blob_config_prod2, web_client, html_bname, expiry_time=False)

        for result_idx, result_row in result_df.iterrows():
            pnum = result_row['PAGE_NUM']
            result_df.loc[result_idx, 'HTML_SAS_URL'] = html_sas_url + '#page' + str(pnum)
        
    prc_running_time = time.time() - prc_str_time
    
    pkl_path_df.loc[idx, 'CT_SECTION_SPLIT_RUNNING_TIME'] = prc_running_time

    try:
        # INSERT RESULT_STG_BACKUP
        backup_query = f"""
            INSERT INTO TB_PREP_RESULT_STG_BACKUP
            (BACKUP_DTTM, IF_CATEGORY_CD, CONTENTS_ID, ITEM_ID, SEQ_ID, FILE_NM, SECTION_NM,
            SECTION_FILE_NM, PAGE_NUM, PAGE_NUMS, FULL_FILE_PATH, HTML_SAS_URL, MODEL_USE_FLAG, PRODUCT_LVL1_CD,
            PRODUCT_LVL2_CD, PRODUCT_LVL3_CD, PROD_MODEL_CD, SALES_MODEL_CD, SYMP_CODE_ONE, SYMP_CODE_TWO, SYMP_CODE_THREE, IF_FLAG, IF_DTTM,
            CREATE_DTTM, CREATE_PGM, UPDATE_DTTM, UPDATE_PGM, RESULT_TEXT, IMAGE_RATIO,
            TEXT_CNT, SIZE_MIN, SIZE_MAX, DOCS_DATE, IF_TYPE_CD)
            SELECT SYSDATE() AS BACKUP_DTTM,
            IF_CATEGORY_CD, CONTENTS_ID, ITEM_ID, SEQ_ID, FILE_NM, SECTION_NM,
            SECTION_FILE_NM, PAGE_NUM, PAGE_NUMS, FULL_FILE_PATH, HTML_SAS_URL, MODEL_USE_FLAG, PRODUCT_LVL1_CD,
            PRODUCT_LVL2_CD, PRODUCT_LVL3_CD, PROD_MODEL_CD, SALES_MODEL_CD, SYMP_CODE_ONE, SYMP_CODE_TWO, SYMP_CODE_THREE, IF_FLAG, IF_DTTM,
            CREATE_DTTM, CREATE_PGM, UPDATE_DTTM, UPDATE_PGM, RESULT_TEXT, IMAGE_RATIO,
            TEXT_CNT, SIZE_MIN, SIZE_MAX, DOCS_DATE, IF_TYPE_CD
            FROM TB_PREP_RESULT_STG
            WHERE 1=1
                AND IF_CATEGORY_CD = '{idx_row['IF_CATEGORY_CD']}'
                AND CONTENTS_ID    = '{idx_row['CONTENTS_ID']}'
                AND ITEM_ID        = '{idx_row['ITEM_ID']}'
            ;
        """
        commit_query(mysql_config, backup_query)
        
        # DELETE RESULT_STG
        delete_query = f"""
            DELETE
            FROM TB_PREP_RESULT_STG
            WHERE 1=1
                AND IF_CATEGORY_CD = '{idx_row['IF_CATEGORY_CD']}'
                AND CONTENTS_ID    = '{idx_row['CONTENTS_ID']}'
                AND ITEM_ID        = '{idx_row['ITEM_ID']}'
            ;
        """
        commit_query(mysql_config, delete_query)
        
        
        # INSERT RESULT_STG
        query = f"""
            INSERT INTO TB_PREP_RESULT_STG
            (IF_CATEGORY_CD, CONTENTS_ID, ITEM_ID, SEQ_ID, FILE_NM, SECTION_NM,
            SECTION_FILE_NM, PAGE_NUM, PAGE_NUMS, FULL_FILE_PATH, HTML_SAS_URL, MODEL_USE_FLAG, PRODUCT_LVL1_CD,
            PRODUCT_LVL2_CD, PRODUCT_LVL3_CD, PROD_MODEL_CD, SALES_MODEL_CD, SYMP_CODE_ONE, SYMP_CODE_TWO, SYMP_CODE_THREE, IF_FLAG, IF_DTTM,
            CREATE_DTTM, CREATE_PGM, UPDATE_DTTM, UPDATE_PGM, RESULT_TEXT, IMAGE_RATIO,
            TEXT_CNT, SIZE_MIN, SIZE_MAX, DOCS_DATE, IF_TYPE_CD)
            VALUES (%s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, FROM_BASE64(%s), %s,
            %s, %s, %s, %s, %s)
            ;
        """
        vals = list(result_df.itertuples(index=False, name=None))
        commit_query_w_vals(mysql_config, query, vals, True)
        
        # UPDATE SOURCE
        update_query = f"""
            UPDATE TB_PREP_SOURCE
            SET PREP_STATUS_CD = 'Y'
                , UPDATE_PGM = 'PAGEHEADER 전처리'
                , UPDATE_DTTM = SYSDATE()
            WHERE 1=1
                AND IF_CATEGORY_CD = '{idx_row['IF_CATEGORY_CD']}'
                AND CONTENTS_ID    = '{idx_row['CONTENTS_ID']}'
                AND ITEM_ID        = '{idx_row['ITEM_ID']}'
            ;
        """
        commit_query(mysql_config, update_query)
    
    except Exception as e:
        error_msg += f"{str(e)}\n"
        
        print('DB INSERT 실패')
        update_query = f"""
            UPDATE TB_PREP_SOURCE
            SET PREP_STATUS_CD = 'E'
                , PREP_ERROR_MSG = '{error_msg}'
                , UPDATE_PGM = 'PAGEHEADER 전처리'
                , UPDATE_DTTM = SYSDATE()
            WHERE 1=1
                AND IF_CATEGORY_CD = '{idx_row['IF_CATEGORY_CD']}'
                AND CONTENTS_ID    = '{idx_row['CONTENTS_ID']}'
                AND ITEM_ID        = '{idx_row['ITEM_ID']}'
            ;
        """
        commit_query(mysql_config, update_query)
    